# Housing Market

### Introduction:

This time we will create our own dataset with fictional numbers to describe a house market. As we are going to create random data don't try to reason of the numbers.

### Step 1. Import the necessary libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import pandas as pd
import numpy as np

spark = SparkSession.builder.appName("Fictitous_names").getOrCreate()

### Step 2. Create 3 differents Series, each of length 100, as follows: 
1. The first a random number from 1 to 4 
2. The second a random number from 1 to 3
3. The third a random number from 10,000 to 30,000

In [3]:
s1 = pd.Series(np.random.randint(1, high=5, size=100, dtype='l'))
s2 = pd.Series(np.random.randint(1, high=4, size=100, dtype='l'))
s3 = pd.Series(np.random.randint(10000, high=30001, size=100, dtype='l'))


### Step 3. Let's create a DataFrame by joinning the Series by column

In [8]:
df = spark.createDataFrame(pd.concat([s1, s2, s3], axis=1))

In [9]:
df.show(10)

+---+---+-----+
|  0|  1|    2|
+---+---+-----+
|  2|  1|26096|
|  4|  1|19341|
|  1|  2|25792|
|  4|  2|28717|
|  3|  3|16124|
|  4|  1|11572|
|  2|  3|21676|
|  1|  2|26105|
|  3|  3|24392|
|  1|  1|29705|
+---+---+-----+
only showing top 10 rows



### Step 4. Change the name of the columns to bedrs, bathrs, price_sqr_meter

In [13]:
houses = df.withColumnRenamed('0', 'bedrs').\
    withColumnRenamed('1', 'bathrs').\
    withColumnRenamed('2', 'price_sqr_meter')

In [14]:
houses.head(5)

[Row(bedrs=2, bathrs=1, price_sqr_meter=26096),
 Row(bedrs=4, bathrs=1, price_sqr_meter=19341),
 Row(bedrs=1, bathrs=2, price_sqr_meter=25792),
 Row(bedrs=4, bathrs=2, price_sqr_meter=28717),
 Row(bedrs=3, bathrs=3, price_sqr_meter=16124)]

### Step 5. Create a one column DataFrame with the values of the 3 Series and assign it to 'bigcolumn'

In [28]:
big_column = houses.select('bedrs').\
    union(houses.select('bathrs')).\
    union(houses.select('price_sqr_meter')).\
    select(F.col('bedrs').alias('all_in_one'))

### Step 6. Oops, it seems it is going only until index 99. Is it true?

In [33]:
big_column.summary().show()

+-------+-----------------+
|summary|       all_in_one|
+-------+-----------------+
|  count|              300|
|   mean|6607.963333333333|
| stddev|9925.887942487141|
|    min|                1|
|    25%|                2|
|    50%|                3|
|    75%|            14542|
|    max|            29888|
+-------+-----------------+



### Step 7. Reindex the DataFrame so it goes from 0 to 299

In [34]:
# na